In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!ls "/content/drive/My Drive/FlowPS/data"

Mounted at /content/drive
GSE18728.csv  GSE25066.csv  GSE50948.csv  GSE9782.csv		  TCGA_LGG.csv
GSE20181.csv  GSE32646.csv  GSE5122.csv   TARGET10.csv		  TCGA_UC.csv
GSE20194.csv  GSE37946.csv  GSE55145.csv  TARGET20Busulfan.csv
GSE20271.csv  GSE39754.csv  GSE59515.csv  TARGET20NoBusulfan.csv
GSE22358.csv  GSE41998.csv  GSE68871.csv  TARGET50.csv
GSE23988.csv  GSE42822.csv  GSE76360.csv  TCGA_LC.csv


Посмотрим на работу алгоритма поиска core marker genes на примере датасета GSE20181:

In [ ]:
df = pd.read_csv('/content/drive/My Drive/FlowPS/change_data/GSE20181.csv')
df.tail()

,Sample ID,Binary response,A1CF,A2M,A4GALT,A4GNT,AAAS,AACS,AADAC,AAGAB,AAK1,AAMDC,AAMP,AANAT,AAR2,AARS,AARSD1,AASDHPPT,AASS,AATF,AATK,ABAT,ABCA1,ABCA11P,ABCA12,ABCA2,ABCA3,ABCA4,ABCA5,ABCA6,ABCA7,ABCA8,ABCB1,ABCB11,ABCB4,ABCB6,ABCB7,ABCB8,ABCB9,ABCC1,...,ZNF835,ZNF839,ZNF84,ZNF85,ZNF862,ZNF91,ZNF93,ZNHIT1,ZNHIT2,ZNHIT3,ZNHIT6,ZNRD1ASP,ZNRF4,ZP2,ZP3,ZPBP,ZPR1,ZRSR2,ZRSR2P1,ZSCAN12,ZSCAN16,ZSCAN18,ZSCAN2,ZSCAN26,ZSCAN31,ZSCAN32,ZSCAN5A,ZSCAN9,ZSWIM1,ZSWIM8,ZSWIM8-AS1,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYX,ZZEF1,ZZZ3
47,48,100,159.745186,559.742486,45.721309,97.223123,66.038053,612.827244,10.884616,46.659486,49.382559,97.814768,273.683995,12.922660,363.800263,600.884191,256.553247,209.938000,5.079213,242.511283,279.201143,281.417194,103.498446,72.618170,51.790919,25.401141,872.719875,60.056364,94.003876,6.101652,146.849113,3.712725,21.977797,14.363940,13.379061,46.514847,156.493559,28.693694,26.332883,79.122014,...,7.642583,123.953406,76.079999,210.251977,36.483695,143.367579,109.401922,517.070846,88.598633,327.918791,128.647925,113.990561,92.214969,7.070247,88.820244,83.949529,153.394960,267.681554,4.742141,13.055714,163.875173,70.539256,12.398714,98.140851,106.538563,124.298361,92.214969,98.223569,35.875569,36.771933,63.621987,205.416134,5.494678,431.953768,15.235929,13.412972,122.214249,190.783192,39.014291,415.800798
48,49,100,829.517515,1609.632889,24.184723,126.986531,16.391632,260.127053,18.260505,22.901147,77.234377,67.365105,238.852339,5.879799,301.030396,1369.488995,172.603326,124.492993,3.169642,296.505089,305.128818,70.674579,196.632406,110.996425,101.559836,42.623990,32.343677,48.274935,100.573299,26.189418,165.162462,98.830779,34.507981,16.734401,13.342149,41.690496,25.014169,25.181767,46.966908,53.452088,...,6.885567,230.857574,61.801804,374.403888,89.315504,253.716272,52.229478,54.222977,20.742559,376.841240,130.714578,285.510673,179.036080,7.328193,19.809914,154.800178,142.226192,409.780355,5.778570,19.167175,93.572277,42.257707,7.718934,9.585303,64.203926,109.610859,144.452615,377.353160,140.289628,32.311490,269.615766,242.547441,13.135400,972.162944,37.184689,42.166478,41.348141,87.135149,20.346048,345.703453
49,50,100,208.440393,1613.350941,24.269641,75.805620,9.407050,341.638940,11.211331,35.504283,59.562700,61.321007,356.048180,8.634132,267.080567,431.190844,65.149035,328.474487,8.475349,225.336461,146.940948,163.474649,236.953126,69.619155,28.449481,24.585010,69.400823,42.291324,102.890125,47.164434,75.805620,630.941003,77.804213,17.264018,22.659177,30.891972,106.428585,17.324333,27.080387,54.426771,...,6.251064,108.282512,112.628031,294.151199,49.250447,267.357854,90.283109,388.805245,29.426296,245.198282,214.774380,200.775278,70.219294,20.175694,222.267131,58.585503,83.932229,310.637130,9.989120,23.500353,123.409362,99.196114,8.853852,141.022165,104.980038,110.281473,68.398886,170.037697,78.822136,36.843130,28.319523,255.952243,6.570268,576.470655,18.189719,22.421760,172.846097,256.274129,28.760924,445.820694
50,51,100,458.914621,1598.074389,41.811811,96.298337,11.855324,220.719409,18.971395,13.600509,49.897787,79.000311,344.494653,8.615376,339.525525,800.770239,105.718531,348.706540,5.909676,148.653904,109.248994,96.820715,231.352668,132.347101,53.489251,30.410094,122.182169,22.955542,92.387294,12.545439,131.100108,49.729984,83.623258,19.126796,26.473716,22.743999,52.334012,31.002643,68.330490,45.617079,...,8.355255,104.519763,114.977267,320.516224,79.273476,374.403888,74.239719,155.975535,17.223150,348.853930,103.415794,280.952916,104.989389,31.214010,19.791510,141.795226,63.792499,297.477729,8.111957,11.472402,159.582730,60.401089,11.111723,61.932313,47.546621,169.891932,114.126727,87.913487,117.139792,25.272285,88.331377,302.997383,13.757607,1204.000648,28.020282,38.789651,40.850153,104.423961,14.517178,383.986309
51,52,100,120.039446,692.579163,69.149450,75.404653,23.158304,366.241145,26.527400,84.648765,53.649245,117.447101,328.291288,7.736591,262.273890,679.910383,247.321906,229.600224,27.416014,211.

In [ ]:
df.shape

(52, 13547)

Разбиваем данные на признаки и таргет:

In [ ]:
X = df.drop(['Sample ID', 'Binary response'], axis=1)
#X = X.iloc[:, :100]

y = df['Binary response']

Запускаем гланый цикл кросс-валидации с перебором всех признаков для каждого объекта датасета(на примере 3 наблюдений):

In [ ]:
from tqdm import tqdm

final = []
for i in tqdm(range(3)):
  res = []
  X2 = X.drop([i])
  y2 = y.drop([i])

  for j in range(len(X2)):
    roc_auc_list = []
    if i != j:
      X3 = X2.drop(j)
      y3 = y2.drop(j)

      for num, k in enumerate(range(X3.shape[1])):

        X_train = X3.iloc[:, k]
        X_train_new = X_train.values.reshape(-1, 1)
          
        roc_auc = roc_auc_score(y3, X_train_new)   
        roc_auc_list.append((num, roc_auc)) 
      
      res.append(roc_auc_list)

  final.append(res)




  0%|          | 0/3 [00:00<?, ?it/s]


 33%|███▎      | 1/3 [11:32<23:04, 692.22s/it]


 67%|██████▋   | 2/3 [23:04<11:32, 692.38s/it]


100%|██████████| 3/3 [34:39<00:00, 693.04s/it]


In [ ]:
len(final)

3

Сохраняем все рок_аук для каждого объекта в файл pickle:

In [ ]:
import pickle

with open('/content/drive/My Drive/FlowPS/features_GSE20181.pickle', 'wb') as f:
    pickle.dump(final, f)

#with open('data.pickle', 'rb') as f:
#    data_new = pickle.load(f)

Отсортируем для каждого датасета без i-го и j-го объекта все признаки по значению roc_auc и выберем TOP 30:

In [ ]:
array = []
for i in range(len(final)):
  arr = []
  for j in range(len(final[i])):
    temp = sorted(final[i][j], key=lambda x: (x[1]), reverse=True)
    arr.append(temp[:30])
  array.append(arr)

Оставим только номера значимых признаков:

In [ ]:
mass = []
for i in range(len(array)):
  mass1 = []
  for j in range(len(array[i])):
    mass2 = []
    for k in range(len(array[i][j])):
      mass2.append(array[i][j][k][0])

    mass1.append(mass2)

  mass.append(mass1)

In [ ]:
#mass[0]

Посмотрим на пересечение всех значимых признаков для каждого i и тем самым получим N наборов core marker features для каждого образца:

In [ ]:
core_marker = []
for i in range(len(mass)):
  temp = list(set.intersection(*map(set, mass[i])))
  core_marker.append(temp)

In [ ]:
print(len(core_marker))
core_marker

3


[[2342, 5548, 8401, 4019, 1876, 4341, 8438, 8825, 7677],
 [640, 1344, 2342, 1512, 5548, 8401, 4019, 1876, 8438],
 [640, 1344, 2342, 5548, 4401, 8401, 4019, 1876, 8438]]